To Try:
* Drop all zero electricity meter readings
* Explore and possibly remove building 1099
* Try cross validation and ensembling of models
* Treat categorical missing / NaNs


* X Remove noisy days for site 0 (See Strategy...notebook on Kaggle)
* X Create time series visualization by site / building / meter type
* X Use meter type as a feature ({0: electricity, 1: chilledwater, 2: steam, 3: hotwater})
* X Add building and site id features (see https://www.kaggle.com/aitude/ashrae-kfold-lightgbm-without-leak-1-08)
    * Set categorical dataset in lgbm fit
* X Research validation strategy and implement
* X 'Primary use' indicator
* X Additional datebased features (month and quarterly indicators, time trends)
* X LightGBM

In [ ]:
import time

import feather
import lightgbm as lgb
from sklearn import model_selection, preprocessing, metrics
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import matplotlib.pyplot as plt

from utils import *

In [ ]:
MAIN = pathlib.Path('/Users/palermopenano/personal/kaggle_energy')
SUBMISSIONS_PATH = MAIN / 'submissions'

# Set Parameters

In [ ]:
sample = False
train_full = True         # False to do KFold CV, True to train on full dataset
create_submission = True  # True to generate submission csv on test

# Number of hours to compute moving average
period = 3
# period = 24

submission_name = 'submission_2019-11-29_remove_zero_electricity_meter_readings'

# Prepare Training Data

In [ ]:
# DNC (does not change)
train = pd.read_csv(MAIN / 'data' / 'train.csv')
train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format=True)

building_metadata = pd.read_csv(MAIN / 'data' / 'building_metadata.csv')

weather_train = pd.read_csv(MAIN / 'data' / 'weather_train.csv')
weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'], infer_datetime_format=True)

In [ ]:
# import seaborn as sns
# df = train
# df = df.loc[(df.building_id == 1099) & (df.meter == 0)]
# sns.lineplot(x='timestamp', y='meter_reading',data=df)
# df

## Compute rolling stat

In [ ]:
# !!!!!!!
# Rolling statistic for weather data
cols_rol = [
    'air_temperature', 
    'dew_temperature',
    'sea_level_pressure',
    'wind_speed'
]

tmp = rolling_stat(
    weather_train, 'timestamp', ['site_id'], 
    cols_rol, period, np.mean
)
weather_train = weather_train.drop(cols_rol, 1)
weather_train = weather_train.merge(tmp, how='inner', on=['site_id', 'timestamp'])

In [ ]:
# Take only a random sample of n buildings
if sample:
    train, randbuilding = df_sample_random_buildings(train, 'building_id', n=5)
    print(randbuilding)

# train = train[train.meter == 0]
print(train.shape)

## Merge in to train

In [ ]:
# DNC
train = train.merge(building_metadata, on='building_id', how='left')
train = train.merge(weather_train, on=['site_id', 'timestamp'], how='left')

print(
    f"Min time {train['timestamp'].min()}",
    f"Max time {train['timestamp'].max()}",
    sep='\n'
)

In [ ]:
# Reduce memory usage
train = reduce_mem_usage(train, cols_exclude=['timestamp'])

# Data Filters

## Remove  first 141 days of meter=0 for site 0
* See discussions in https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113054#latest-675811
* No changes need to be made to the test set as this only concerns the meter_reading

In [ ]:
print("Before filter:", train.shape)
first141d_site0_meter0_cond = \
    (train.site_id == 0) & \
    (train.meter == 0) & \
    (train.timestamp.dt.dayofyear >= 0) & \
    (train.timestamp.dt.dayofyear <= 141)

train = train.loc[~first141d_site0_meter0_cond,:]
print("After filter:", train.shape)

In [ ]:
# Exploration code

# df = train.loc[
#     (train.site_id == 0) & (train.meter == 0), 
#     ['building_id','meter','timestamp','meter_reading']]
# df = df.sort_values('timestamp', ascending=True)
# bs_site0 = df.building_id.unique()
# daysofyear = df.timestamp.dt.dayofyear.unique()

# for d in daysofyear:
#     cond = df.timestamp.dt.dayofyear == d
# #     print(df.loc[cond, 'meter_reading'].head())
#     print(d, df.loc[cond, 'meter_reading'].median(), df.loc[cond, 'meter_reading'].mean())

# print(bs_site0)
# for b in bs_site0:
#     cond = (df.timestamp.dt.dayofyear >= 0) & \
#            (df.timestamp.dt.dayofyear <= 141) & \
#            (df.building_id == b)
    
#     median = df.loc[cond, 'meter_reading'].median()
#     mean = df.loc[cond, 'meter_reading'].mean()
#     print(b, median, mean)

# import seaborn as sns
# for b in bs_site0:
#     print(b)
#     cond = (df.timestamp.dt.dayofyear >= 0) & \
#            (df.timestamp.dt.dayofyear <= 141) & \
#            (df.building_id == b)
#     sns.lineplot(x='timestamp',y='meter_reading',data=df[cond])
#     plt.show()

In [ ]:
# Remove all zero or missing electricity meter readings
print("Before remove zero electiricity meter reading filter:", train.shape)
zero_meter_reading_cond = \
    (train.meter == 0) & \
    (train.meter_reading == 0.0)
train = train.loc[~zero_meter_reading_cond, :]
print("After remove zero electiricity meter reading filter:", train.shape)
print("--------------")
print("Before remove na filter:", train.shape)
train = train.loc[~train.meter_reading.isna(), :]
print("After remove na filter:", train.shape)

# Feature Engineering

In [ ]:
# Feature engineering: take log of square_feet
train['square_feet'] = np.log1p(train['square_feet'])

In [ ]:
# Feature engineering: Add datebased features
# Monday is 0
# If dayofweek is 5 or 6, then it is a weekend
# // is "floored" division (i.e. 6//5 is equal to 1, 3//5 is 0)

add_datepart(
    train, 'timestamp', datetimeformat=None,
    drop=False, time=True, errors="raise"
)
train["weekend"] = train["timestamp"].dt.weekday // 5

In [ ]:
# Feature engineering: precip_depth_1
# Convert -1 and NaN precipitation to 0
# Create trace rain indicator
# Create NaN indicator

def precip_depth_1_hr_FE(df, m):
    df['precip_depth_1_hr_nan'] = df['precip_depth_1_hr'].isna()
    
    if m:
        df.loc[df['precip_depth_1_hr'].isna(), 'precip_depth_1_hr'] = m
    else:
        m = df['precip_depth_1_hr'].median()
        df.loc[df['precip_depth_1_hr'].isna(), 'precip_depth_1_hr'] = m

    df['precip_depth_1_hr_isTrace'] = (df['precip_depth_1_hr'] == -1)
    df.loc[df['precip_depth_1_hr'] == -1, 'precip_depth_1_hr'] = 0
    return df, m

train, precip_m = precip_depth_1_hr_FE(train, m=None)
# train[['precip_depth_1_hr_nan', 'precip_depth_1_hr_isTrace', 'precip_depth_1_hr']]

In [ ]:
# Feature engineering: wind_direction
# Replace nan with median wind_directin angle
# Create nan indicator
# Convert to sine and cosine features

def wind_direction_FE(df, m=None):
    df['wind_direction_nan'] = df['wind_direction'].isna()

    if m:
        df.loc[df['wind_direction'].isna(), 'wind_direction'] = m
    else:
        m = df['wind_direction'].median()
        df.loc[df['wind_direction'].isna(), 'wind_direction'] = m

    df['wind_direction_sin'] = np.sin(np.radians(df['wind_direction']))
    df['wind_direction_cos'] = np.cos(np.radians(df['wind_direction']))
    return df, m

train, wind_direction_m = wind_direction_FE(train, m=None)
# train[['wind_direction_nan','wind_direction_sin','wind_direction_cos','wind_direction']]

In [ ]:
# Feature engineering: primary_use
# Apply label encoder

le = LabelEncoder()
train['primary_use'] = le.fit_transform(train['primary_use'])

## Transformations on Meter Reading (outcome variable)

In [ ]:
# DNC
train['meter_reading'] = np.where(
    train['meter_reading']>=0, train['meter_reading'], 0)

In [ ]:
# Smooth out time series moving average

cols_rol_y = ['meter_reading']
tmp = rolling_stat(
    train, 'timestamp', ['building_id', 'meter'], 
    cols_rol_y, period, np.mean
)

train = train.drop(cols_rol_y, 1)
train = train.merge(tmp, how='inner', on=['building_id', 'meter', 'timestamp'])

# Shift back by an hour because moving average tends to shift the series forward
train['meter_reading'] = train.groupby(['building_id','meter'])['meter_reading'].shift(-1)

# Replace missing with mean for building / site
train['meter_reading'] = (
    train.
    groupby(['building_id', 'meter'])['meter_reading'].
    transform(lambda x: x.fillna(x.mean()))   
)

In [ ]:
y = np.log1p(train['meter_reading'])
print(y.ndim, y.shape, y.min(), y.max())
print(y.describe().apply(lambda x: format(x, ',.2f')))

In [ ]:
# import seaborn as sns

# dd = train.loc[
#     (train.building_id == 544) & 
#     (train.meter == 0)
# ]
# d1 = dd.loc[
#     (dd.timestamp >= '2016-10-08') & 
#     (dd.timestamp <= '2016-10-10'), 
#     ['meter_reading', 'timestamp']]

# d2 = dd.loc[
#     (dd.timestamp >= '2016-10-08') & 
#     (dd.timestamp <= '2016-10-10'), 
#     ['meter_reading_avg','timestamp']]

# sns.lineplot(x='timestamp', y='meter_reading', data=d1)
# sns.lineplot(x='timestamp', y='meter_reading_avg', data=d2)

## Define Features to Include in Training

In [ ]:
cont_feats = [
        'square_feet',
        'floor_count',
        'air_temperature',
        'dew_temperature',
        'sea_level_pressure',
        'wind_speed',
        'precip_depth_1_hr',
        'precip_depth_1_hr_nan', 
        'precip_depth_1_hr_isTrace',
]

cat_feats = [
    'timestampDayofweek',
    'primary_use',
    'year_built',
    'timestampMonth',
#     'timestampWeek',
    'timestampHour',
    'weekend',
    'site_id',
    'building_id',
    'meter'
]

In [ ]:
if not train['timestamp'].is_monotonic_increasing:
    raise Exception(
        "timestamp should be sorted in increasing order "
        "for KFold validation to work properly"
    )

In [ ]:
# DNC
train = train[cont_feats + cat_feats]
print(
    f"Training on {train.shape[0]} records",
    f"Number of features: {train.shape[1]}",
    sep='\n'
)

# Impute Missing

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')  # CHANGED
imputed_df = pd.DataFrame(imp.fit_transform(train))
imputed_df.columns = train.columns
train = imputed_df

print("Saving train...")
feather.write_dataframe(train, MAIN / 'train')

In [ ]:
# Save final dataset before training here so that 
# entire pipeline need not be run when switching between
# validation and full training

train = feather.read_dataframe(MAIN / 'train')

# KFold CV (Unshuffled)
Variation of cv approach in 

https://www.kaggle.com/kimtaegwan/what-s-your-cv-method?scriptVersionId=22371767

evaluated only on the second fold, since validation set for this are from a time period after the training set. Note disadvantage of current implementation of this approach: missing imputation by mean of a feature leaks into the validation set

In [ ]:
if not train_full:

    folds = 2

    kf = model_selection.KFold(
        n_splits=folds, shuffle=False, random_state=42)

    for fold_, (trn_idx, val_idx) in enumerate(kf.split(train, y)):

        # Skip first fold to avoid worst data leakage
        # due to all training set time > validation set time
        if fold_ == 0:
            continue

        print(fold_, trn_idx.shape, val_idx.shape)

        # Note potential leakage here if missing imputation is done before 
        # before this cell
        tr_x, tr_y = train.iloc[trn_idx], y[trn_idx]
        vl_x, vl_y = train.iloc[val_idx], y[val_idx]

        reg = lgb.LGBMRegressor(
            learning_rate=0.05,
            boosting="gbdt",
            n_estimators=3000,
            feature_fraction=.7,
            min_child_weight=3,
            subsample=0.6,
            colsample_bytree=.9,
            objective='regression',
            metric='rmse',
            n_jobs=8,
            seed=27,
            num_leaves=40
        )

        reg.fit(
            tr_x, tr_y,
            eval_set=[(vl_x, vl_y)],
            early_stopping_rounds=200,
            verbose=100,
            categorical_feature=cat_feats
        )

        gc.collect()

# Train on Full Dataset

In [ ]:
# Train on full sample for submission
if train_full:

    print("Training on entire training dataset")
    # Number of estimators based on KFold CV results
    n_estimators_cv = 500

    reg = lgb.LGBMRegressor(
        learning_rate=0.05,
        boosting="gbdt",
        n_estimators=n_estimators_cv,
        feature_fraction=.7,
        min_child_weight=3,
        subsample=0.6,
        colsample_bytree=.9,
        objective='regression',
        metric='rmse',
        n_jobs=8,
        seed=27,
        num_leaves=40
    )
    reg.fit(
        train, y,
        categorical_feature=cat_feats
    )
    
    # Save model
    reg.booster_.save_model(str(SUBMISSIONS_PATH / f'model_{submission_name}.txt'))
    
#     reg = LinearRegression()
#     reg.fit(train, y)

# Define Dataset to Evaluate

In [ ]:
if create_submission:
    # Evaluate on test set
    test = pd.read_csv(MAIN / 'data' / 'test.csv')
    test['timestamp'] = pd.to_datetime(test['timestamp'])

    weather_test = pd.read_csv(MAIN / 'data' / 'weather_test.csv')
    weather_test['timestamp'] = pd.to_datetime(weather_test['timestamp'])

# Apply Evaluation Set Transformations

In [ ]:
if create_submission:
    tmp = rolling_stat(
        weather_test, 'timestamp', ['site_id'], 
        cols_rol, period, np.mean
    )
    weather_test = weather_test.drop(cols_rol, 1)
    weather_test = weather_test.merge(tmp, how='inner', on=['site_id', 'timestamp'])

    # DNC
    # Merge into training
    test = test.merge(building_metadata, on='building_id', how='left')
    test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
    
    if sample:
        test = test[test['building_id'].isin(randbuilding)]

    print("Apply date operation...")
    add_datepart(
        test, 'timestamp', datetimeformat=None,
        drop=False, time=True, errors="raise"
    )
    test["weekend"] = test["timestamp"].dt.weekday // 5

    # Apply feature engineering to test set
    print("Apply feature engineering and imputed values...")
    test,_ = precip_depth_1_hr_FE(test, m=precip_m)
    test, _ = wind_direction_FE(test, m=wind_direction_m)
    test['primary_use'] = le.transform(test['primary_use'])  # CHANGED

    # Remove binding from namespace
    # and force release of memory
    del building_metadata, weather_train
    gc.collect()

    test = test[cont_feats + cat_feats + ['row_id']]
    test['square_feet'] = np.log1p(test['square_feet'])
    
    # Apply missing imputation used in training
    test_v = test.drop('row_id', 1).values
    test_v = imp.transform(test_v)
    test_v.shape
    
    # !!! Save and load prepared test set
    feather.write_dataframe(test, MAIN / 'test')

# Generate Submission Scores

In [ ]:
if create_submission:
    print("Generating submission")
    
    test['meter_reading'] = np.expm1(reg.predict(test_v))

    print("Clipping meter reading to zero...")
    # Save predictions as a column in a df
    # Clip to a min of 0 and infinity (a_max is None)
    test['meter_reading'] = np.clip(test['meter_reading'].values, 0, None)
    
    print("Copying subset of dataframe...")
    sample_submission = test[['row_id', 'meter_reading']].copy()

    print("Recasting to float32 and rounding values...")
    sample_submission.loc[:,'meter_reading'] = (
        sample_submission.loc[:, 'meter_reading'].
        astype('float32').
        round(2)
    )
    sample_submission.loc[:,'row_id'] = (
        sample_submission.loc[:, 'row_id'].
        astype('int32')
    )

    print("Saving csv...")
    sample_submission.to_csv(SUBMISSIONS_PATH / (submission_name + '.csv'), index=False)

    print(sample_submission.shape)
    print(sample_submission['meter_reading'].min(), sample_submission['meter_reading'].max())

# Visualize predictions
Need

* Trained model
* training predictions with timestamp, building_id, and meter
* test prediciton with timestamp, building_id, and meter

# End

In [ ]:
print("End!")